In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-07-30 14:37:20--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   122MB/s    in 0.2s    

2023-07-30 14:37:21 (122 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

Clean the data

In [ ]:
# Get all isbns with more than 100 ratings and Users with more than 200 ratings
filteredIsbn = df_ratings['isbn'].value_counts()
filteredUsers = df_ratings['user'].value_counts()

filteredUsers = filteredUsers[filteredUsers >= 200]
filteredIsbn = filteredIsbn[filteredIsbn >= 100]

# Print the initial shape of our merged dataframe
cleaned_df = df_books.merge(df_ratings, on='isbn')
print(cleaned_df.shape, 'initial merged shape')
# Filter the merged dataframe by ISBNs with 100 or more ratings
cleaned_df = cleaned_df[cleaned_df['isbn'].isin(filteredIsbn.index)]
print(cleaned_df.shape, 'after cleaning books')
# Filter the merged dataframe by users with 200 or more ratings
cleaned_df = cleaned_df[cleaned_df['user'].isin(filteredUsers.index)]
print(cleaned_df.shape, 'after cleaning users')
# Drop duplicates from users that have multiple ratings for the same book but different isbns
cleaned_df = cleaned_df.drop_duplicates(['title', 'user'])
print(cleaned_df.shape, 'after dropping duplicates')




(1031175, 5) initial merged shape
(137423, 5) after cleaning books
(49517, 5) after cleaning users
(49136, 5) after dropping duplicates


In [ ]:
# Pivot the dataframe with book titles as the index, users as columns and ratings per user as values
pivot_df = cleaned_df.pivot(index='title', columns='user', values='rating')

# Fill NA values with zeros
pivot_df = pivot_df.fillna(0)
print(pivot_df.tail())

user                                                254     2276    2766    \
title                                                                        
Without Remorse                                        0.0     0.0     0.0   
Year of Wonders                                        0.0     0.0     0.0   
You Belong To Me                                       0.0     0.0     0.0   
Zen and the Art of Motorcycle Maintenance: An I...     0.0     0.0     0.0   
\O\" Is for Outlaw"                                    0.0     0.0     0.0   

user                                                2977    3363    4017    \
title                                                                        
Without Remorse                                        0.0     0.0     0.0   
Year of Wonders                                        7.0     0.0     0.0   
You Belong To Me                                       0.0     0.0     0.0   
Zen and the Art of Motorcycle Maintenance: An I...     0.0     

In [ ]:
# Make a matrix out of user ratings
matrix = csr_matrix(pivot_df.values)

In [ ]:
#Create a NearestNeighbors model with cosine metric and leave everything as default
model = NearestNeighbors(metric='cosine', algorithm='auto')
# Fit the model to our matrix data
model.fit(matrix)

NearestNeighbors(metric='cosine')

In [ ]:
bookIndex = pivot_df.index.get_loc("Where the Heart Is (Oprah's Book Club (Paperback))")
pivot_df.iloc[bookIndex].values.reshape(1, -1)

array([[ 0.,  0.,  0.,  0., 10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  8.,
         8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,
         0.,  0.,  0.,  0.,  0.,  9.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  8.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  8.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  # Get the index number by searching for it with the book title
  bookIndex = pivot_df.index.get_loc(book)
  # Call the model with a flattened array of the books ratings and declare to show the 6 highest matches (one being the book itself)
  distances, indices = model.kneighbors(pivot_df.iloc[bookIndex].values.reshape(1, -1), n_neighbors = 6)
  # Empty array to put the results into
  recommended_books = []

  # Loop through the resulting distances
  for i in range(len(distances[0])):
    # Append the searched books title first and append  the list for results afterwards
    if i == 0:
      recommended_books.append(pivot_df.index[bookIndex])
      recommended_books.append([])
    else:
      # Insert into the result list at the front since the distances list will be sorted from lowest to highest
      recommended_books[1].insert(0, [pivot_df.index[indices[0][i]], distances[0][i]])



  return recommended_books

print(get_recommends('The Queen of the Damned (Vampire Chronicles (Paperback))'))

['The Queen of the Damned (Vampire Chronicles (Paperback))', [['Catch 22', 0.7939835], ['The Witching Hour (Lives of the Mayfair Witches)', 0.74486566], ['Interview with the Vampire', 0.73450685], ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.53763384], ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.51784116]]]


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
